## El objetivo de este ejercicio es calcular la función de valor de una política dada usando sampleo Monte Carlo.
<img src="Prediccion_MC.PNG">

In [ ]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict

if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [ ]:
env = BlackjackEnv()

In [ ]:
def mc_prediction(policy, env, num_episodes, discount_factor=1.0):
    """
    Algoritmo de predicción Monte Carlo "primera-visita". Calcula la función de valor para una 
    politica dada usando sampleo.
    
    Argumentos:
        policy: Política, función que mapea observaciones a probabilidades de cada acción.
        env: Ambiente de OpenAI gyn.
        num_episodes: Número de episodios a samplear.
        discount_factor: factor de descuento gama.
    
    Retorna:
        Un diccionario que mapea estado -> valor.
        El estado es una tupla y el valor es un float.
    """

    # Mantiene una suma y conteo de los retornos de cada estado
    # para calcular un promedio. Podría usarse un array para guardar todos los retornos
    # pero sería ineficiente en memoria.
    # CUIDADO: las llaves de un diccionario deben ser inmutables, por ejemplo tuples
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # El valor final de la función de valor
    V = defaultdict(float)
    
    # Implementar!
    # CONSEJOS:
    # loop num_episodes veces
    for i in range(num_episodes):
        episodio = []
        estados_en_episodio = []
        estado = env.reset()
        # generar un episodio de máximo 100 pasos (o parar cuando acabe el episodio)
        # un episodio es una lista de tuplas (estado, accion, recompensa)
        while True:
            accion = policy(estado)
            proximo_estado, recompensa, fin, _ = env.step(accion)
            estados_en_episodio.append(estado)
            episodio.append((estado, accion, recompensa))
            if fin:
                break
            estado = proximo_estado

        # con el episodio generado: 
        # iterar sobre los estados visitados en este episodio, 
        # TIP 1: podemos usar set() para obtener estados unicos
        estados_unicos = set(estados_en_episodio)
        for estado in estados_unicos:
            # por cada estado encontrar su primera aparición en el episodio (estamos implementando "primera-visita")
            primer_indice = estados_en_episodio.index(estado)
            # obtener el retorno a partir de ese momento
            G = 0 
            for j in range(primer_indice, len(episodio)):
                tupla = episodio[j]
                G += tupla[2]*discount_factor**(j-primer_indice)
            # acumular los retornos si queremos usar la formula (1) para el promedio
            #returns_sum[estado] += G
            returns_count[estado] += 1 
            #V[estado] = returns_sum[estado] / returns_count[estado] # formula (1) para el promedio
            V[estado] = V[estado] + 1/returns_count[estado] * (G - V[estado]) # formula alternativa para el promedio
    return V    

In [ ]:
def sample_policy(observation):
    """
    Una política que se planta si el jugador tiene un puntaje > 20 y pide otra carta caso contrario.
    """
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

In [ ]:
V_10k = mc_prediction(sample_policy, env, num_episodes=10000)
plotting.plot_value_function(V_10k, title="10,000 Episodios")

V_500k = mc_prediction(sample_policy, env, num_episodes=500000)
plotting.plot_value_function(V_500k, title="500,000 Episodios")